### **imports**

In [27]:
import pandas as pd
import numpy as np
from astropy.io import ascii
from astropy.table import QTable, Table, Column
from astropy import units as u
import string
import upsilon

### **making a table of all of the stars in the txt file**

In [2]:
column1 = []
column2 = []
column3 = []
column4 = []

with open('rrlyr_vsx_clean.txt', 'r') as file:
    for line in file:
        empty = []
        components = line.split(' ', 3)
    
        column1.append(components[0])
        column2.append(float(components[1]))
        column3.append(float(components[2]))
        column4.append(components[3].strip())

# column2 = column2 * u.deg
# column3 = column3 * u.deg

final_table = QTable([column1, column2, column3, column4], names = ("Star Type", "RA", "Dec", "Other Info"))

In [3]:
final_table

Star Type,RA,Dec,Other Info
str24,float64,float64,str30
RRAB,0.01307,-2.43007,PS1-3PI J000003.12-022548.2
RRAB,0.01521,35.36286,GM And
RRAB,0.01669,18.40698,CSS_J000004.0+182425
RRAB,0.02771,56.65331,V1023 Cas
RRAB,0.03171,34.67397,SERIV 27
RRAB,0.03901,5.58986,CSS_J000009.3+053523
RRAB,0.04274,-2.51629,PS1-3PI J000010.25-023058.6
RRAB,0.04331,-21.92086,CSS 120828:000010-215515
RRAB,0.04782,62.71524,Gaia DR2 430046902041301376


### **importing the asassn client**

In [10]:
from pyasassn.client import SkyPatrolClient	
client = SkyPatrolClient()

Welcome to ASAS-SN Skypatrol!

Current Deployment Version: 0.6.17 (26 JAN 2024)
Please upgrade your client if not up to date.



### **downloading a csv file with the data from the star at a specified index**

In [20]:
folder_path = "/Users/zoesurles/Desktop/variable_star_classification/downloaded_lightcurves/"

In [39]:
def classification_model(path, ra, dec, vsx_id, asassn_id):
  # Load a classification model.
  # (takes time, do only once if processing multiple lcs)
  rf_model = upsilon.load_rf_model()


  ############# Read input lc data #############
  # Read the input lightcurve file
  f = open(path, 'r')
  data = np.loadtxt(f, dtype={'names': ('JD', 'mag', 'err'),'formats': ('float64', 'float64', 'float64')})
  f.close()

  # Split it into three arrays
  date=data['JD']
  mag=data['mag']
  err=data['err']

  # Sigma-clip the input lightcurve
  date, mag, err = upsilon.utils.sigma_clipping(date, mag, err,  threshold=3, iteration=1)

  ############# Extract lightcurve features #############
  e_features = upsilon.ExtractFeatures(date, mag, err)
  e_features.run()
  features = e_features.get_features()

  ############# Classify #############
  # Classify the light curve
  label, probability, flag = upsilon.predict(rf_model, features)

  # Print results

  output_file = open(path.replace('.txt', '_classification.txt'), 'w')
  output_file.write("\n\n######### Classification results #########\n")
  output_file.write("flag = " + str(flag) + " (0 - classification success, 1 - suspicious period)\n")
  output_file.write("class = " + str(label) + "\n")
  output_file.write("class_probability = " + str(probability) + "\n")
  output_file.write("position = " + str(ra) + " " + str(dec) + "\n")
  output_file.write("vsx_id = " + str(vsx_id) + "\n")
  output_file.write("asassn_id = " + str(asassn_id) + "\n")
  output_file.close()

  print("\n\n######### Classification results #########\n")
  print("flag = " + str(flag) + " (0 - classification success, 1 - suspicious period)")
  print("class = " + str(label) )
  print("class_probability = " + str(probability) )
  print("\n\n######### Lightcurve features #########\n")

  for feature in features:
    print( str(feature) + " = " + str(features[feature]) )
  #  print feature," = ",features[feature]

In [41]:
num_iterations = 5
empty_list = []
for i in range(num_iterations):
    search = client.cone_search(ra_deg = final_table["RA"][i], dec_deg = final_table["Dec"][i], radius=0.0003, catalog='master_list', download=True)

    savecsv = search.save(save_dir = folder_path, file_format="csv")[1]
    empty_list.append(savecsv)

    new_file = pd.read_csv(savecsv, skiprows=1)

    g_filter = new_file[new_file['phot_filter'] == 'g']
    web_format = g_filter[["jd", "mag", "mag_err"]]
    web_format.to_csv(savecsv.replace(".csv", ".txt"), sep='\t', index=False, header = False)

    classification_model(savecsv.replace(".csv", ".txt"), final_table["RA"][i], final_table["Dec"][i], final_table["Other Info"][i], new_file["asas_sn_id"][0])

Pulled 1 of 1


/Users/zoesurles/miniconda3/lib/python3.11/site-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator DecisionTreeClassifier from version 0.24.1 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/Users/zoesurles/miniconda3/lib/python3.11/site-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator RandomForestClassifier from version 0.24.1 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(




######### Classification results #########

flag = 0 (0 - classification success, 1 - suspicious period)
class = RRL_ab
class_probability = 0.88


######### Lightcurve features #########

amplitude = 0.33576400320037264
cusum = 0.13057498116667463
eta = 1.8372295759321078
hl_amp_ratio = 0.7415104682932441
kurtosis = -0.5529804139751473
n_points = 896
period = 0.4936273709306785
period_SNR = 106.25647083917974
period_log10FAP = -47.92730871672018
period_uncertainty = 0.00010553607329547021
phase_cusum = 0.2202706923195133
phase_eta = 1.4730283715912005
phi21 = 0.2547161606071446
phi31 = 1.2291766835529498
quartile31 = 0.731348768429612
r21 = 0.43460568255198984
r31 = 0.2713391933460415
shapiro_w = 0.9767159223556519
skewness = -0.38398688043539236
slope_per10 = -0.008432668821191371
slope_per90 = 0.007441519285166582
stetson_k = 0.6638592376996094
weighted_mean = 17.29078943759845
weighted_std = 0.4382569858916419
Pulled 1 of 1


/Users/zoesurles/miniconda3/lib/python3.11/site-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator DecisionTreeClassifier from version 0.24.1 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/Users/zoesurles/miniconda3/lib/python3.11/site-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator RandomForestClassifier from version 0.24.1 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(




######### Classification results #########

flag = 0 (0 - classification success, 1 - suspicious period)
class = RRL_ab
class_probability = 0.97


######### Lightcurve features #########

amplitude = 0.3432799329059622
cusum = 0.1452776746557756
eta = 2.5032121771222386
hl_amp_ratio = 0.3752540817315546
kurtosis = -0.8239731458557591
n_points = 548
period = 0.7067830026597548
period_SNR = 136.323076028037
period_log10FAP = -79.52952372617999
period_uncertainty = 0.00025304222789990227
phase_cusum = 0.3270529745835165
phase_eta = 0.3073737948909103
phi21 = 0.7616105986457724
phi31 = 4.672091901915031
quartile31 = 0.5005662300894382
r21 = 0.36856251711608334
r31 = 0.2740004452797228
shapiro_w = 0.8901732563972473
skewness = -0.6636568203032644
slope_per10 = -0.08311948503485267
slope_per90 = 0.12945126232614182
stetson_k = 0.7491324538122004
weighted_mean = 13.092348996917007
weighted_std = 0.28710942582204807
Pulled 1 of 1


...

/Users/zoesurles/miniconda3/lib/python3.11/site-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator DecisionTreeClassifier from version 0.24.1 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/Users/zoesurles/miniconda3/lib/python3.11/site-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator RandomForestClassifier from version 0.24.1 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(




######### Classification results #########

flag = 1 (0 - classification success, 1 - suspicious period)
class = EB_ESD
class_probability = 0.31


######### Lightcurve features #########

amplitude = 0.0601404883964961
cusum = 0.37323090609784637
eta = 1.0055800906390062
hl_amp_ratio = 0.6194317122533876
kurtosis = 0.09621984998459254
n_points = 453
period = 0.9997052328546377
period_SNR = 69.33980963044654
period_log10FAP = -28.93092743668683
period_uncertainty = 0.0005705151224009719
phase_cusum = 0.21278000820042667
phase_eta = 1.4281972905939377
phi21 = -3.558202959582374
phi31 = -4.577157850963905
quartile31 = 0.08298446379010471
r21 = 0.3182324743746061
r31 = 0.08538793151690534
shapiro_w = 0.9841779470443726
skewness = -0.36802234080393875
slope_per10 = -0.30710423050666635
slope_per90 = 0.2943987099835373
stetson_k = 0.724940070331493
weighted_mean = 14.878064779943065
weighted_std = 0.05833948138669103
Pulled 1 of 1


/Users/zoesurles/miniconda3/lib/python3.11/site-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator DecisionTreeClassifier from version 0.24.1 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(




######### Classification results #########

flag = 0 (0 - classification success, 1 - suspicious period)
class = RRL_ab
class_probability = 1.0


######### Lightcurve features #########

amplitude = 0.47169546291458164
cusum = 0.309062937555019
eta = 1.8669104190563452
hl_amp_ratio = 0.3124018295480004
kurtosis = -0.4193325941458528
n_points = 543
period = 0.46197284192571736
period_SNR = 125.8601819428668
period_log10FAP = -65.98128593791847
period_uncertainty = 9.267258987280536e-05
phase_cusum = 0.32929803539105806
phase_eta = 0.4682973095720427
phi21 = 0.518777939233599
phi31 = 1.3643490162560412
quartile31 = 0.6198908129808256
r21 = 0.42847678584203114
r31 = 0.2842978299081692
shapiro_w = 0.9052363038063049
skewness = -0.8032336837892786
slope_per10 = -0.026789237936296863
slope_per90 = 0.033965564312094486
stetson_k = 0.6232290966226436
weighted_mean = 15.890647032534282
weighted_std = 0.44133324832001314


/Users/zoesurles/miniconda3/lib/python3.11/site-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator RandomForestClassifier from version 0.24.1 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
